# Core Model Guide: MPCController

This notebook provides a guide to the `MPCController`, an advanced control model in the CHS SDK that uses **Model Predictive Control**.

## 1. Theoretical Background

Model Predictive Control (MPC) is a modern control strategy that uses an explicit model of the system it is controlling (the "plant") to make predictions about the system's future behavior.

At each time step, the MPC controller performs the following actions:
1.  **Measures** the current state of the system.
2.  **Predicts** the system's future behavior over a certain time window, called the **Prediction Horizon (P)**, using its internal model.
3.  **Optimizes** a sequence of future control moves over a **Control Horizon (M)** to minimize an objective function. The objective function typically penalizes deviations from the setpoint and excessive control effort.
4.  **Executes** only the *first* control move from the optimized sequence.
5.  **Repeats** the entire process at the next time step (this is known as a receding horizon strategy).

MPC is powerful because it can handle constraints (e.g., max pump flow) and can proactively respond to forecasted disturbances.

## 2. API and Parameters

The `MPCController` in the CHS SDK is a linear MPC, meaning it linearizes its internal model at each step to create a convex optimization problem.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.control.mpc_controller import MPCController
from water_system_sdk.src.chs_sdk.modules.modeling.storage_models import LinearTank

The `MPCController` is initialized with:

- `prediction_model` (BaseModel): An instance of a model that the MPC will use for its internal predictions. **This is the most important parameter.**
- `prediction_horizon` (int): The number of future steps (P) to predict.
- `control_horizon` (int): The number of future control moves (M) to optimize.
- `set_point` (float): The target value.
- `q_weight` (float): The weight on the setpoint deviation penalty.
- `r_weight` (float): The weight on the control effort penalty.
- `u_min`, `u_max` (float): Constraints on the control output.

## 3. Code Example: Controlling a Tank with MPC

We will use the same `LinearTank` system from the PID tutorial, but this time we will control it with MPC. This allows for a direct comparison.

In [ ]:
# 1. Create the "real world" system to be controlled
real_tank = LinearTank(area=1000.0, initial_level=2.0)

# 2. Create a separate, identical model for the MPC to use for prediction
prediction_model_for_mpc = LinearTank(area=1000.0, initial_level=2.0)

# 3. Initialize the MPC Controller
mpc_controller = MPCController(
    prediction_model=prediction_model_for_mpc,
    prediction_horizon=10, # Predict 10 steps into the future
    control_horizon=3,     # Optimize the next 3 control moves
    set_point=10.0,        # Target level is 10m
    q_weight=1.0,          # Penalty on deviation from setpoint
    r_weight=0.5,          # Penalty on control effort (to prevent aggressive changes)
    u_min=0.0              # Inflow cannot be negative
)

# 4. Simulation Setup
dt = 1.0
n_steps = 200
history = {"time": [], "level": [], "inflow": []}

# 5. Simulation Loop
for t in range(n_steps):
    # Get current state from the "real" tank
    current_level = real_tank.level
    
    # MPC calculates the optimal control action
    # We are not providing a disturbance forecast in this simple example
    control_output = mpc_controller.step(current_state=current_level)
    
    # Apply the control action to the real tank
    real_tank.input.inflow = control_output
    real_tank.step(dt=dt)
    
    # Record results
    history["time"].append(t)
    history["level"].append(real_tank.level)
    history["inflow"].append(control_output)

print("Simulation complete.")

## 4. Visualization

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Level
color = 'tab:blue'
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Water Level (m)', color=color)
ax1.plot(history['time'], history['level'], color=color, label='Water Level (MPC)')
ax1.axhline(y=10.0, color='r', linestyle='--', label='Setpoint (10.0m)')
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='lower right')
ax1.grid(True)

# Plot control action (inflow)
ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Control Inflow (m³/s)', color=color)
ax2.plot(history['time'], history['inflow'], color=color, linestyle=':', label='MPC Control Action')
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')

plt.title('MPC Control of Reservoir Water Level')
fig.tight_layout()
plt.show()

### Analysis of Results

The plot shows that the `MPCController` also successfully brings the water level to the setpoint. Compared to a well-tuned PID, the MPC's response can be smoother and less prone to overshoot because it can "see" into the future. By adjusting the weights `q_weight` and `r_weight`, you can trade off between how quickly the system reaches the setpoint versus how much control effort (e.g., energy for pumping) is used.